Import Libraries

In [ ]:
import urllib

Mounting with S3

In [ ]:
Access_key = "****"
Secret_key = "****"
Encoded_secret_key = urllib.parse.quote(Secret_key, "")
Bucket_name = "my-cluster87235"
Mount_name = "mount1"
dbutils.fs.mount("s3a://%s:%s@%s" %(Access_key, Encoded_secret_key, Bucket_name), "/mnt/%s" % Mount_name)

Checking the Mounted Data

In [ ]:
dbutils.fs.ls("/mnt/mount1")

In [ ]:
dbutils.fs.head("/mnt/mount1/listings.csv")

Command to take Help from Methods

In [ ]:
dbutils.fs.help()

Creating A DataFrame

In [ ]:
File_loc = "dbfs:/mnt/mount1/listings.csv"
File_type = "csv"

row_header = "true"
delimeter = ","
infer_Schema = "true"

df1 = spark.read.format(File_type) \
    .option("header", row_header) \
    .option("inferSchema", infer_Schema) \
    .option("delimeter", delimeter) \
    .load(File_loc)

In [ ]:
display(df1)

Print Schema

In [ ]:
df1.printSchema()

In [ ]:
df1.tail(1000)

In [ ]:
df1.select("id", "name", "price").show()

In [ ]:
from pyspark.sql.functions import *

Type Casting the Data Type

In [ ]:
df2 = df1.withColumn("price", col("price").cast("int"))

In [ ]:
df2.printSchema()

In [ ]:
display(df1.select("id", "name", "price"))

Create a Internal Table

In [ ]:
df2.write.mode("overwrite").format("parquet").saveAsTable("listing")

Create a External Table

In [ ]:
df2.write.mode("overwrite").format("parquet").option("path", "/user/external_tables/").saveAsTable("listing2")


In [ ]:
%sql
select * from listing

In [ ]:
%sql
select * from listing2

SQL Queries

In [ ]:
%sql

select id, name, host_id, host_name, price from listing2
where price > (select avg(price) from listing2)

In [ ]:
%sql

select id, name, host_id, host_name, price from listing2
where price > 1000

DataFrame Queries

In [ ]:
df_price = df1.where(col("price") > 1000)

In [ ]:
df_price2 = df_price.select("id","name","host_id","host_name", "price").show()

In [ ]:
df_price_min_nights = df.where((col("price") > 2500) & (col("minimum_nights") > "5")) \
                       .select("id","name","host_id","host_name", "price", "minimum_nights").show()

In [ ]:
from pyspark.sql.functions import *

Writing Transformed data to AWS S3

In [ ]:
df_price_min_nights.repartition(1).write.csv("dbfs:/mnt/mount1/list_price_min_ngt_data")

In [ ]:
df_price2.repartition(1).write.csv("dbfs:/mnt/mount1/listing_fixed")

Checking the data if Available

In [ ]:
dbutils.fs.head("dbfs:/mnt/mount1/listing_fixed/")

Unmounting Databricks with S3

In [ ]:
dbutils.fs.unmount("/mnt/mount1")

In [ ]:
-------------------------------------------------Thank You---------------------------------------